In [1]:
from LoopDataConverter import LoopConverter, InputData, SurveyName
from osgeo import gdal, osr
import os 

In [2]:
def create_raster(output_path, bbox, epsg, pixel_size, value=100):
    minx, miny, maxx, maxy = bbox
    cols = int((maxx - minx) / pixel_size)
    rows = int((maxy - miny) / pixel_size)
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_path, cols, rows, 1, gdal.GDT_Byte)
    out_raster.SetGeoTransform([minx, pixel_size, 0, maxy, 0, -pixel_size])
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg)
    out_raster.SetProjection(srs.ExportToWkt())
    out_band = out_raster.GetRasterBand(1)
    out_band.Fill(value)
    out_band.FlushCache()
    out_raster = None

In [3]:
xmin, ymax = 449200.89,7322334.93
xmax, ymin = 463912.6,7315529.2
zmin, zmax = -3000, 3000
bounding_box = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}
f_path = "/home/rabii/Git_Repos/LoopDataConverter/test_data/NTGS_data"
create_raster(
    os.path.join(f_path, "DEM.tif"),
    (bounding_box['minx'], bounding_box['miny'], bounding_box['maxx'], bounding_box['maxy']),
    7854,
    1000,
)

In [2]:
xmin, ymax = 449200.89,7322334.93
xmax, ymin = 463912.6,7315529.2
zmin, zmax = -3000, 3000
bbox_3d = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}

xmin, ymax = -24.210915,134.499656
xmax, ymin = -24.272464,134.644287
zmin, zmax = -3000, 3000
bbox_3d = {
    "minx": xmin,
    "miny": ymin,
    "maxx": xmax,
    "maxy": ymax,
    "base": zmin,
    "top": zmax,
}

In [5]:
input_data = InputData(GEOLOGY="LoopDataConverter/test_data/NTGS_data/NAM_LithOutcrop_100K.shp", 
                       STRUCTURE="test_data/NTGS_data/NAM_StructData_100K.shp",
                       FAULT="test_data/NTGS_data/NAM_Fault_100K.shp",
                       FOLD="test_data/NTGS_data/NAM_Fold_100K.shp")

loop_converter = LoopConverter(survey_name=SurveyName.NTGS,
                               data=input_data)
loop_converter.convert()

DriverError: LoopDataConverter/test_data/NTGS_data/NAM_LithOutcrop_100K.shp: No such file or directory

In [3]:
import elevation

In [4]:
_path = "/home/rabii/Git_Repos/LoopDataConverter/test_data/NTGS_data/DEM.tif"
elevation.clip(bounds=(xmin, ymin, xmax, ymax), output=_path)


make: Entering directory '/home/rabii/.cache/elevation/SRTM1'
make: Nothing to be done for 'download'.
make: Leaving directory '/home/rabii/.cache/elevation/SRTM1'
make: Entering directory '/home/rabii/.cache/elevation/SRTM1'
gdalbuildvrt -q -overwrite SRTM1.vrt 
Usage: gdalbuildvrt [-tileindex field_name]
                    [-resolution {highest|lowest|average|user}]
                    [-te xmin ymin xmax ymax] [-tr xres yres] [-tap]
                    [-separate] [-b band]* [-sd subdataset]
                    [-allow_projection_difference] [-q]
                    [-addalpha] [-hidenodata]
                    [-srcnodata "value [value...]"] [-vrtnodata "value [value...]"] 
                    [-ignore_srcmaskband]
                    [-a_srs srs_def]
                    [-r {nearest,bilinear,cubic,cubicspline,lanczos,average,mode}]
                    [-oo NAME=VALUE]*
                    [-input_file_list my_list.txt] [-overwrite]
                    [-strict | -non_strict]
    

ERROR 1: No input dataset specified.
make: *** [Makefile:14: SRTM1.vrt] Error 1


CalledProcessError: Command 'make -C /home/rabii/.cache/elevation/SRTM1 all ' returned non-zero exit status 2.